<a href="https://colab.research.google.com/github/Karla-Flores/Big-Data--Challenge/blob/main/big_data2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [67.4 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hi

In [3]:
# For connection to Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-09-24 02:05:15--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.51MB/s    in 0.2s    

2021-09-24 02:05:16 (4.51 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# ** DATA 1**

In [5]:
# Read in data
# Instructions mention that Tab ('\t')
from pyspark import SparkFiles
url='https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Luggage_v1_00.tsv.gz'
spark.sparkContext.addFile(url)
review_df = spark.read.csv(SparkFiles.get('amazon_reviews_us_Luggage_v1_00.tsv.gz'), sep='\t', header=True, inferSchema=True)

# Show DataFrame
review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   40884699| R9CO86UUJCAW5|B00VGTN02Y|     786681372|Teenage Mutant Ni...|         Luggage|          3|            0|          0|   N|                Y|my review of this...|my review of this...| 2015-08-31|
|         US|   23208852|R3PR8X6QGVJ8B1|B005KIWL0E|     618251799|Kenneth Cole Reac...|         Luggage|          5|    

In [6]:
# Count
print(review_df.count())

348657


In [7]:
# Drop null values
dropna_df = review_df.dropna()
dropna_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   40884699| R9CO86UUJCAW5|B00VGTN02Y|     786681372|Teenage Mutant Ni...|         Luggage|          3|            0|          0|   N|                Y|my review of this...|my review of this...| 2015-08-31|
|         US|   23208852|R3PR8X6QGVJ8B1|B005KIWL0E|     618251799|Kenneth Cole Reac...|         Luggage|          5|    

In [8]:
# Count
print(dropna_df.count())

348613


In [9]:
# CREATE TABLE review_id_table (
#   review_id TEXT PRIMARY KEY NOT NULL,
#   customer_id INTEGER,
#   product_id TEXT,
#   product_parent INTEGER,
#   review_date DATE -- this should be in the formate yyyy-mm-dd
# );

In [10]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only columns with active users
review_id_table_df =dropna_df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
review_id_table_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R9CO86UUJCAW5|   40884699|B00VGTN02Y|     786681372| 2015-08-31|
|R3PR8X6QGVJ8B1|   23208852|B005KIWL0E|     618251799| 2015-08-31|
|R39BO2819ABUPF|   17100246|B007UNSHJ6|     810480328| 2015-08-31|
|R3ADL7V6EGGEEP|   13533670|B00WHFNXB4|      20765193| 2015-08-31|
|R1OXYPBPLVRMI5|   38541771|B000M5NBYU|     812890709| 2015-08-31|
|R1WYM8Z5ATQ98O|   12686499|B00ACBJ89G|     624906499| 2015-08-31|
|R3LCIANTN1H9EC|   27767206|B001H4BQ1A|     385044506| 2015-08-31|
|R3U2M23N1P0KQ6|   46630606|B00ESG6NDK|      51297652| 2015-08-31|
|R29MB6N7HB6NZI|   41622754|B00M14SAXC|     832113872| 2015-08-31|
| RGEQ6DGRG7DQG|   15296380|B012PC5QAY|     490793867| 2015-08-31|
|R38HNH0BSS2KFE|   21206415|B00VWKWWMG|     356411837| 2015-08-31|
|R2CATRM9CH59LY|   34554097|B00N2U14UK|     326835930| 2015-08

In [11]:
print(review_id_table_df.count())

348613


In [12]:
# -- This table will contain only unique values
# CREATE TABLE products (
#   product_id TEXT PRIMARY KEY NOT NULL UNIQUE,
#   product_title TEXT
# );

In [13]:
# This table will contain only unique values
products_df = dropna_df.select(['product_id', 'product_title']).distinct()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B005GUQCGK|Everest Gym Bag w...|
|B00M6O0WQK|Sprayground Unise...|
|B00UFFW5XK|Polare Italian Le...|
|B00J8O730E|Nicole Lee Shoppi...|
|B0028Y67PI|David King & Co. ...|
|B00DHCUWD8|PealRa Banana Lea...|
|B00O9QORAK|Luxebell Luggage ...|
|B0125Q648I|Shacke 2 x Metal ...|
|B004URKJY6|Small Orange Sol ...|
|B004R6B5MU|New Maxam 16 Inch...|
|B00FB4X9TY|Rockland Luggage ...|
|B00IDSDI1E|Kobwa(TM) Yellow ...|
|B00AE0NW5Q|Pacsafe Venturesa...|
|B00YNP622S|Great White Shark...|
|B00QF5Q7KG|Lewis N Clark Bal...|
|B0109ZA54Q|Passport Holder- ...|
|B00KYFPMWC|ECOSUSI Vintage C...|
|B005H4D4TI|CalPak Hollywood ...|
|B0053GFAJC|ShedRain Umbrella...|
|B00PY00AWY|Unisex Casual Gen...|
+----------+--------------------+
only showing top 20 rows



In [14]:
print(products_df.count())

64584


In [15]:
# -- Customer table for first data set
# CREATE TABLE customers (
#   customer_id INT PRIMARY KEY NOT NULL UNIQUE,
#   customer_count INT
# );

In [16]:
# -- Customer table for first data set
counts_df = dropna_df.groupBy('customer_id').count().orderBy('customer_id')
counts_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|      10302|    1|
|      10408|    1|
|      10736|    1|
|      10953|    1|
|      11023|    3|
|      11049|    1|
|      11106|    1|
|      11266|    1|
|      11340|    1|
|      11584|    1|
|      11586|    1|
|      11720|    1|
|      11799|    1|
|      11850|    2|
|      11880|    1|
|      12085|    1|
|      12099|    1|
|      12199|    1|
|      12371|    1|
|      12433|    1|
+-----------+-----+
only showing top 20 rows



In [17]:
# Renaming column
customers_df = counts_df.withColumnRenamed('count', 'customer_count')
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|      10302|             1|
|      10408|             1|
|      10736|             1|
|      10953|             1|
|      11023|             3|
|      11049|             1|
|      11106|             1|
|      11266|             1|
|      11340|             1|
|      11584|             1|
|      11586|             1|
|      11720|             1|
|      11799|             1|
|      11850|             2|
|      11880|             1|
|      12085|             1|
|      12099|             1|
|      12199|             1|
|      12371|             1|
|      12433|             1|
+-----------+--------------+
only showing top 20 rows



In [18]:
print(counts_df.count())

308151


In [19]:
# -- vine table
# CREATE TABLE vine_table (
#   review_id TEXT PRIMARY KEY,
#   star_rating INTEGER,
#   helpful_votes INTEGER,
#   total_votes INTEGER,
#   vine TEXT
# );

In [20]:
# -- vine table
vine_table_df = dropna_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R9CO86UUJCAW5|          3|            0|          0|   N|
|R3PR8X6QGVJ8B1|          5|            0|          0|   N|
|R39BO2819ABUPF|          4|            0|          0|   N|
|R3ADL7V6EGGEEP|          4|            0|          0|   N|
|R1OXYPBPLVRMI5|          5|            0|          0|   N|
|R1WYM8Z5ATQ98O|          3|            0|          0|   N|
|R3LCIANTN1H9EC|          4|            1|          1|   N|
|R3U2M23N1P0KQ6|          5|            0|          0|   N|
|R29MB6N7HB6NZI|          1|            2|          2|   N|
| RGEQ6DGRG7DQG|          5|            0|          0|   N|
|R38HNH0BSS2KFE|          1|            1|          1|   N|
|R2CATRM9CH59LY|          5|            4|          4|   N|
|R18DMKNDPJ1BSN|          5|            0|          1|   N|
| RZRSHBWXO3XUQ|          5|            

In [21]:
print(vine_table_df.count())

348613


# Write DataFrame to RDS

In [ ]:
# Configuration 
mode = "append"
jdbc_url="jdbc:postgresql://database-1.cotqj8kvxlpi.us-east-1.rds.amazonaws.com:5432/aws"
config = {"user":"postgres", 
          "password": "postgres", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to active_user table in RDS
review_id_table_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write dataframe to billing_info table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write dataframe to billing_info table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
 # Write DataFrame to active_user table in RDS
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)